## Extinction Strain Rate with Sensitivity Alalysis

Run with Cantera version 2.3.0 or higher.<BR>
Written by Alon Grinberg Dana, inspired by:<BR>
(1) <a href="https://github.com/Cantera/cantera-jupyter/blob/master/flames/twin_premixed_flame_axisymmetric.ipynb">twin_premixed_flame_axisymmetric.ipynb</a> in Cantera, and<BR>
(2) <a href="http://www.cantera.org/docs/sphinx/html/cython/examples/onedim_diffusion_flame_extinction.html"> diffusion_flame_extinction.py</a> (cite: <a href="http://dx.doi.org/10.1155/2014/484372">doi:10.1155/2014/484372</a>)

**See comments and instructions at the bottom of this notebook**

## Import modules

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function
from __future__ import division
import matplotlib.pyplot as plt
from shutil import copyfile
from sys import stdout
import cantera as ct
import pandas as pd
import numpy as np
import operator
import pylab
import time
import os
# delete if working outside Jupyter:
%matplotlib notebook

print("Running Cantera Version: " + str(ct.__version__))

Running Cantera Version: 2.3.0a3


## User variables

In [4]:
initialAxialVelocity = 200                     # cm/s; Set the initial inlet axial velocity

model = 'mechs/xo1247R.cti'                    # relative path to the mechanism input file (.cti form)
composition = 'CH4(1):0.4615, O2(2):1, N2(3):3.76'

name = "SA"                             # name will be used for the output files

To = 300                # K, default:  300
Po = ct.one_atm * 1     # atm
#Po = 101325            # Pa, default: 101325

half_width = 0.7        # Define a domain half-width x cm wide (i.e., the whole domain is 2x cm wide)

topSA = 10     # default: 10 (choose the top sensitive reactions to display, enter 0 to skip SA altogether)
#topSA = 10   # default: 10 (choose the top sensitive reactions to display, enter 0 to skip SA altogether)
dk = 1.05     # default: 1.02 pertubation multiplier for Ai

# number of grid points; set to 0 to be determined automatically by the solver
#(auto is recommended at first since it's fast, but not as accurate as having a large grid)
grid_pnts = 0  # default: 0 for auto, 1000 for uniform
refine = True  # defauld: True (recommended) ; enable grid refinement?

verboseLevel = 3 # default: 3 (1 essentials, 2 all, 3 also figs; one lvl up for SA; in any case all is saved to files)

species1 = ''    # plot concentration profiles. Leave blank ('') to repress plotting
species2 = ''

tran = 'Multi' # default: 'Multi'; choose transport calculation method: 'Mix' for mixture-averaged or 'Multi' for multi-component

# Define a limit for the maximum temperature below which the flame is considered as extinguished
temperature_limit_extinction = 1000  # K, default: 1000
# convergence creterion for the change in max T
delta_T_min = 0.5  # K, default: 0.5

# Troubleshooting parameters
delta_alpha_low_boundary = 1e-15 # default: 1e-15 (If a solution cannot be obtained by reducing delta alpha)
max_n = 200 # default: 200 (max 'normal' iterations before speeding things up)
# The axial velocity will be multiplied by AxialVelocityIncreaseFactor * (vIncCount + delta_v_factor)
# if extinction takes too long to achieve (determined by n_max)
# Or divided by AxialVelocityDecreaseFactor * (vDecCount + delta_v_factor)
# if ignition can't be achieved.
# note: AxialVelocityDecreaseFactor and AxialVelocityIncreaseFactor should be different to avoid going back and forth
axialVelocityIncreaseFactor = 2.5 # default: 2.5
axialVelocityDecreaseFactor = 2 # default: 2
u_vIncCount = 1 # initialAxialVelocity aux increase factor
u_vDecCount = 1 # initialAxialVelocity aux decrease factor
delta_v_factor = 0.2 # default: 0.2 (increment of vIncCount and vDecCount every time the axial velocity needs a boost)

# Set normalized initial strain rate
u_alpha = [1.]                # default: [1.] (has to be a list)
# Initial relative strain rate increase
u_delta_alpha = 1.            # default: 1.
# Factor of refinement of the strain rate increase
delta_alpha_factor = 50.      # default: 50.
# convergence creterion for delta_alpha
delta_alpha_min = 1E-3        # default: 1E-3; recommended values: 1E-1, 1E-3, 1E-5, 1E-6

# set tolerances
steady_tolerances = [1.0e-5, 1.0e-12]    # relative and absolute, default: [1.0e-5, 1.0e-12]
transient_tolerances = [1.0e-4, 1.0e-9]  # relative and absolute, default: [5.0e-4, 1.0e-11]

# SET CRITERIA FOR GRID REFINEMENT
ratio = 2.0   # default: 2.0  (10.0 in Cantera)
slope = 0.25  # default: 0.3  (0.8 in Cantera), (0.0 < slope < 1.0)
curve = 0.25  # default: 0.3  (0.8 in Cantera), (0.0 < curve < 1.0)
prune = 0.03  # default: 0.05 (0.0 in Cantera) Set << 'slope' or 'curve'; zero to disable pruning the grid

# Exponents for the initial solution variation with changes in strain rate
# Taken from Fiala and Sattelmayer (2014)
exp_d_a = - 1. / 2.   # default: - 1. / 2.
exp_u_a = 1. / 2.     # default: 1. / 2.
exp_V_a = 1.          # default: 1.
exp_lam_a = 2.        # default: 2.
exp_mdot_a = 1. / 2.  # default: 1. / 2.

# data directory; If you run this script in parallel from the same directory,
# it is recommended to give a different data directory name to each instance
u_data_directory = 'ESR_data/' # default: 'ESR_data/'

## Functions and definitions

In [11]:
def initializer(v):
    alpha = u_alpha
    delta_alpha = u_delta_alpha
    gas = ct.Solution(model)
    gas.TPX = To, Po, composition
    width = half_width * 0.01
    # Create the flame object
    if grid_pnts == 0:
        oppFlame = ct.CounterflowTwinPremixedFlame(gas, width=width)
    else:
        grid = np.linspace(0.0, width, grid_pnts)
        oppFlame = ct.CounterflowTwinPremixedFlame(gas, grid=grid)
    oppFlame.reactants.mdot = gas.density * v * 0.01 # units kg/m2/s
    oppFlame.transport_model = tran
    # Define relative and absolute error tolerances
    oppFlame.flame.set_steady_tolerances(default=steady_tolerances)
    oppFlame.flame.set_transient_tolerances(default=transient_tolerances)
    oppFlame.set_refine_criteria(ratio=ratio, slope=slope, curve=curve, prune=prune)
    return gas, oppFlame, alpha, delta_alpha, width

# Differentiation function; used to compute normal strain-rate
def derivative(x, y):
    dydx = np.zeros(y.shape, y.dtype.type)
    dx = np.diff(x)
    dy = np.diff(y)
    dydx[0:-1] = dy/dx
    dydx[-1] = (y[-1] - y[-2])/(x[-1] - x[-2])
    return dydx

def computeStrainRates(oppFlame):
    # Compute the derivative of axial velocity to obtain normal strain rate
    strainRates = derivative(oppFlame.grid, oppFlame.u)
    # Obtain the Characteristic Strain Rate (K)
    n = 0
    while strainRates[n] > strainRates[n+1]:
        n +=1
    strainRatePoint = n
    K = abs(strainRates[n])
    return strainRates, strainRatePoint, K

def computeConsumptionSpeed(oppFlame):
    Tb = max(oppFlame.T)
    Tu = min(oppFlame.T)
    rho_u = max(oppFlame.density)
    integrand = oppFlame.heat_release_rate/oppFlame.cp
    I = np.trapz(integrand, oppFlame.grid)
    Sc = I/(Tb - Tu)/rho_u
    return Sc

def column(matrix, i):
    return [row[i] for row in matrix]

def solver(gas, oppFlame, alpha, delta_alpha, width, t0):
    n = -1                         # Iteration indicator
    tracker = []                   # Tracking: n, u0, ESR, Tmax
    t = -1                         # tracker index
    n_last_burning = 0             # Indicator of the latest flame still burning
    T_max = [np.max(oppFlame.T)]   # List of peak temperatures
    axialVelocity = [initialAxialVelocity]
    S1 = []
    S2 = []
    restoreSwitch = 0              # indicate the number of times troubleshooting was attempted
    SArestoreFile = 1              # save restore file at the first time delta_alpha is reduced for SA continuation
    vIncCount = u_vIncCount
    vDecCount = u_vDecCount
    log = []                       # log troubleshooting messages for output file
    print("Solving %s with u0 = %6.1f cm/s:"%(stripModelName(model),100*oppFlame.reactants.mdot/gas.density))
    while True:
        strainRates, strainRatePoint, K = computeStrainRates(oppFlame)
        n += 1
        # try solving
        try:
            oppFlame.solve(loglevel=0, refine_grid=refine, auto=True)
        except Exception as e:
            if verboseLevel >= 2:
                print("\nError: Did not converge at n = %3d. alpha = %8.2e,   delta_alppha = %8.2e,   Tmax = %8.1f K"%(
                        n, alpha[-1], delta_alpha, np.max(oppFlame.T)))
        # if still burning: save, append, check for convergence
        if np.max(oppFlame.T) > temperature_limit_extinction:
            n_last_burning = n
            t += 1
            tracker.append([n, np.max(oppFlame.u)*100, K, np.max(oppFlame.T)])
            if verboseLevel >=1:
                t1 = time.time() - t0
                m, s = divmod(t1, 60)
                h, m = divmod(m, 60)
                d, h = divmod(h, 24)
                stdout.write("\rn = %3d,     u = %8.1f cm/s,     K = %8.1f 1/s,     Tmax = %8.1f K,     elaped time: %02d %02d:%02d:%02d"%(
                        tracker[t][0],tracker[t][1],tracker[t][2],tracker[t][3],d, h, m, s))
                stdout.flush()
            file_name = 'extinction_{0:04d}.xml'.format(n)
            oppFlame.save(data_directory + file_name, name='solution', loglevel=0,
                          description='Cantera version ' + ct.__version__ +
                          ', reaction mechanism ' + model)
            T_max.append(np.max(oppFlame.T))
            # CONVERGED?
            #print("\n%6.1f - %6.1f = %5.1f <? %4.2f   &   %f <? %f\n"%(T_max[-2],T_max[-1],T_max[-2] - T_max[-1],delta_T_min,delta_alpha,delta_alpha_min))
            if ((T_max[-2] - T_max[-1] < delta_T_min) &
                    (delta_alpha < delta_alpha_min)):
                if verboseLevel >= 2:
                    print('\nFlame (almost) extinguished at n = {0}.'.format(n),
                          'Abortion criterion satisfied.')
                break
            if n > max_n: # if the solver is having a hard time, restart and increase the initial axial velocity
                print('n is high')
                n = 0
                n_last_burning = 0
                axialVelocity.append(axialVelocity[-1] * axialVelocityIncreaseFactor * vIncCount)
                gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                if verboseLevel >= 2:
                    print("\nThis seems to be taking too long... Changing initial axial velocity from %6.1f to %6.1f at n=%d (TrSh1)"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                log.append("\nChanged initial axial velocity from %6.1f to %6.1f at n=%d"%(
                        axialVelocity[-2],axialVelocity[-1],n))
                vIncCount += delta_v_factor
        else: # if extinct
            if delta_alpha >= delta_alpha_low_boundary:
                delta_alpha /= delta_alpha_factor # Reduce relative strain rate increase
                if verboseLevel >= 2:
                    print("\nFlame extinguished at n = %3d. Restoring n = %3d with alpha = %8.2e and delta_alpha = %8.2e (E2)"%(
                          n, n_last_burning, alpha[n_last_burning], delta_alpha))
                if n_last_burning > 0:
                    # Restore last burning solution
                    file_name = 'extinction_{0:04d}.xml'.format(n_last_burning)
                    oppFlame.restore(data_directory + file_name, name='solution', loglevel=0)
                    if SArestoreFile: # save restore file for SA continuation on first reduction of delta_alpha
                        if n_last_burning > 2:
                            file_name0 = 'extinction_{0:04d}.xml'.format(n_last_burning-2)
                        elif n_last_burning == 2:
                            file_name0 = 'extinction_{0:04d}.xml'.format(n_last_burning-1)
                        else:
                            file_name0 = 'extinction_{0:04d}.xml'.format(n_last_burning)
                        copyfile(data_directory + file_name0, data_directory + 'restoreSA.xml')
                        SAalpha = [alpha[n_last_burning-2]]
                        SAdelta_alpha = delta_alpha * delta_alpha_factor
                        SArestoreFile = 0
                else: # haven't ignited yet (n_last_burning = 0)
                    n = 0
                    axialVelocity.append(axialVelocity[-1] / (axialVelocityDecreaseFactor * vDecCount))
                    gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                    if verboseLevel >= 2:
                        print("\nCouldn't ignite. Changing initialAxialVelocity from %6.1f to %6.1f at n=%d (TrSh2)"%(
                                axialVelocity[-2],axialVelocity[-1],n))
                    log.append("\nChanged initialAxialVelocity from %6.1f to %6.1f at n=%d"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                    vDecCount -= delta_v_factor
            else: # if delta_alpha is too low, troubleshoot
                if (grid_pnts != 0) & (restoreSwitch == 0):
                    oppFlame = ct.CounterflowTwinPremixedFlame(gas, width=width) #switch to an automated grid size
                    if verboseLevel >= 2:
                        print("\nCouldn't converge by reducint delta alpha, switching to an automated grid size to better estimate the temperature")
                        print("\nRestoring delta_alpha to 0.02 (TrSh3)")
                    log.append("\nSwitched to an automated grid size")
                    delta_alpha = 0.02 # restore delta alpha to a realistic value
                    log.append("\nRestoring delta_alpha to 0.02")
                if (restoreSwitch == 1) & (n_last_burning > 1):
                    n_last_burning -= 1
                    log.append("\nReducing n_last_burning by 1")
                    if verboseLevel >= 2:
                        print("Couldn't converge, reducing n last burning by 1")
                        print("\nFlame extinguished at n = %3d. Restoring n = %3d with alpha = %8.2e and delta_alpha = %8.2e (TrSh4)"%(
                          n, n_last_burning, alpha[n_last_burning], delta_alpha))
                    # Restore last burning solution
                    file_name = 'extinction_{0:04d}.xml'.format(n_last_burning)
                    oppFlame.restore(data_directory + file_name, name='solution', loglevel=0)
                if n > max_n:
                    n = 0
                    n_last_burning = 0
                    axialVelocity.append(axialVelocity[-1] * axialVelocityDecreaseFactor * vDecCount)
                    gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                    if verboseLevel >= 2:
                        print("\nThis seems to be taking too long... Let's initialize & reduce the velocity")
                        print("\nChanging initialAxialVelocity from %6.1f to %6.1f at n=%d (TrSh5)"%(
                                axialVelocity[-2],axialVelocity[-1],n))
                    log.append("\nInitialized")
                    log.append("\nChanged initialAxialVelocity from %6.1f to %6.1f at n=%d"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                    vDecCount -= delta_v_factor
                restoreSwitch += 1 # indicate that the number of times troubleshooting was attempted, so different cases could be tried out
        # Update relative strain rates
        alpha.append(alpha[n_last_burning] + delta_alpha)
        strain_factor = alpha[-1] / alpha[n_last_burning]
        normalized_grid = oppFlame.grid / (oppFlame.grid[-1] - oppFlame.grid[0])
        # Update mass fluxes
        oppFlame.reactants.mdot *= strain_factor ** exp_mdot_a
        # Update velocity
        oppFlame.set_profile('u', normalized_grid, oppFlame.u * strain_factor ** exp_u_a)
        # Update pressure curvature
        oppFlame.set_profile('lambda', normalized_grid, oppFlame.L * strain_factor ** exp_lam_a)
    strainRates, strainRatePoint, K = computeStrainRates(oppFlame)
    return K, strainRatePoint, log, n_last_burning, alpha, delta_alpha, grid_pnts, SAalpha, SAdelta_alpha, tracker


def SAsolver(gas, oppFlame, alpha, delta_alpha, width, t0):
    oppFlame.restore(data_directory + 'restoreSA.xml', name='solution', loglevel=0)
    n = -1
    T_max = [np.max(oppFlame.T)]
    n_last_burning = 0
    restoreSwitch = 0
    vIncCount = u_vIncCount
    vDecCount = u_vDecCount
    log = []
    while True:
        strainRates, strainRatePoint, K = computeStrainRates(oppFlame)
        n += 1
        try:
            oppFlame.solve(loglevel=0, refine_grid=refine, auto=True)
        except Exception as e:
            if verboseLevel >= 3:
                print('\nError: Did not converge at n = %3d. alpha = %8.2e,   delta_alppha = %8.2e,   Tmax = %8.1f K'%(
                        n, alpha[-1], delta_alpha, np.max(oppFlame.T)))
        if np.max(oppFlame.T) > temperature_limit_extinction:
            n_last_burning = n
            if verboseLevel >=2:
                t1 = time.time() - t0
                m, s = divmod(t1, 60)
                h, m = divmod(m, 60)
                d, h = divmod(h, 24)
                stdout.write("\rn = %3d,     u = %8.1f cm/s,     K = %8.1f 1/s,     Tmax = %8.1f K,     elaped time: %02d %02d:%02d:%02d"%(
                        n, np.max(oppFlame.u)*100, K, np.max(oppFlame.T),d, h, m, s))
                stdout.flush()
            file_name = 'extinction_{0:04d}.xml'.format(n)
            oppFlame.save(data_directory + file_name, name='solution', loglevel=0,
                          description='Cantera version ' + ct.__version__ +
                          ', reaction mechanism ' + model)
            T_max.append(np.max(oppFlame.T))
            # CONVERGED?
            if ((T_max[-2] - T_max[-1] < delta_T_min) &
                    (delta_alpha < delta_alpha_min)):
                if verboseLevel >= 3:
                    print('\nFlame (almost) extinguished at n = {0}.'.format(n),
                          'Abortion criterion satisfied.')
                break
            if n > max_n:
                print('n is high')
                n = 0
                n_last_burning = 0
                axialVelocity.append(axialVelocity[-1] * axialVelocityIncreaseFactor * vIncCount)
                gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                if verboseLevel >= 2:
                    print("\nThis seems to be taking too long... Changing initial axial velocity from %6.1f to %6.1f at n=%d (TrSh1)"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                log.append("\nChanged initial axial velocity from %6.1f to %6.1f at n=%d"%(
                        axialVelocity[-2],axialVelocity[-1],n))
                vIncCount += delta_v_factor
        else: # if extinct
            if delta_alpha >= delta_alpha_low_boundary:
                delta_alpha /= delta_alpha_factor # Reduce relative strain rate increase
                if verboseLevel >= 3:
                    print("\nFlame extinguished at n = %3d. Restoring n = %3d with alpha = %8.2e and delta_alpha = %8.2e (E2)"%(
                          n, n_last_burning, alpha[n_last_burning], delta_alpha))
                if n_last_burning > 0:
                    # Restore last burning solution
                    file_name = 'extinction_{0:04d}.xml'.format(n_last_burning)
                    oppFlame.restore(data_directory + file_name, name='solution', loglevel=0)
                else: # haven't ignited yet (n_last_burning = 0)
                    n = 0
                    axialVelocity.append(axialVelocity[-1] / (axialVelocityDecreaseFactor * vDecCount))
                    gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                    if verboseLevel >= 3:
                        print("\nCouldn't ignite. Changing initialAxialVelocity from %6.1f to %6.1f at n=%d (TrSh2)"%(
                                axialVelocity[-2],axialVelocity[-1],n))
                    log.append("\nChanged initialAxialVelocity from %6.1f to %6.1f at n=%d"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                    vDecCount += delta_v_factor
            else: # if delta_alpha is too low, troubleshoot
                if (restoreSwitch == 0):
                    oppFlame = ct.CounterflowTwinPremixedFlame(gas, width=width) #switch to an automated grid size
                    if verboseLevel >= 3:
                        print("\nCouldn't converge by reducint delta alpha, Restoring delta_alpha to 0.02 (TrSh3)")
                    log.append("\nRestored delta_alpha to 0.02")
                    delta_alpha = 0.02 # restore delta alpha to a realistic value
                if (restoreSwitch == 1) & (n_last_burning > 1):
                    n_last_burning -= 1
                    log.append("\nReduced n_last_burning by 1")
                    if verboseLevel >= 3:
                        print("Couldn't converge, reducing n last burning by 1")
                        print("\nFlame extinguished at n = %3d. Restoring n = %3d with alpha = %8.2e and delta_alpha = %8.2e (TrSh4)"%(
                          n, n_last_burning, alpha[n_last_burning], delta_alpha))
                    file_name = 'extinction_{0:04d}.xml'.format(n_last_burning)
                    oppFlame.restore(data_directory + file_name, name='solution', loglevel=0)
                if n > max_n:
                    n = 0
                    n_last_burning = 0
                    axialVelocity.append(axialVelocity[-1] * axialVelocityDecreaseFactor * vDecCount)
                    gas, oppFlame, alpha, delta_alpha, width = initializer(axialVelocity[-1])
                    if verboseLevel >= 3:
                        print("\nThis seems to be taking too long... Let's initialize & reduce the velocity")
                        print("\nChanging initialAxialVelocity from %6.1f to %6.1f at n=%d (TrSh5)"%(
                                axialVelocity[-2],axialVelocity[-1],n))
                    log.append("\nInitialized")
                    log.append("\nChanged initialAxialVelocity from %6.1f to %6.1f at n=%d"%(
                            axialVelocity[-2],axialVelocity[-1],n))
                    vDecCount -= delta_v_factor
                restoreSwitch += 1 # indicate that the number of times troubleshooting was attempted, so different cases could be tried out
        alpha.append(alpha[n_last_burning] + delta_alpha)
        strain_factor = alpha[-1] / alpha[n_last_burning]
        normalized_grid = oppFlame.grid / (oppFlame.grid[-1] - oppFlame.grid[0])
        oppFlame.reactants.mdot *= strain_factor ** exp_mdot_a
        oppFlame.set_profile('u', normalized_grid, oppFlame.u * strain_factor ** exp_u_a)
        oppFlame.set_profile('lambda', normalized_grid, oppFlame.L * strain_factor ** exp_lam_a)
    strainRates, strainRatePoint, K = computeStrainRates(oppFlame)
    return K, log


def printResults(K,oppFlame,density,Sc,d,h,m,s):
    print('---------------------------------------------------------------------------')
    print('Parameters at the extinction point:')
    print('EXTINCTION STRAIN RATE = %8.1f 1/s'%(K))
    print('Tmax = {0:6.1f} K'.format(np.max(oppFlame.T)))
    print('P = {0} bar'.format(oppFlame.P / 1e5))
    print('Axial velocity = %8.1f cm/s'%(np.max(oppFlame.u)*100))
    print("Consumption Speed: {0:.2f} cm/s".format(100*Sc))
    print("Elaped time for normal solution (w/o SA): %02d:%02d:%02d"%(h, m, s))
    print("\n\nTotal elapsed time without SA: %02d days and %02d:%02d:%02d"%(d, h, m, s))
    
    
def stripModelName(model):
    a=i=0
    d=-1
    for c in model:
        if c == '/':
            a=i+1
        elif c == '.':
            d=i
        i+=1
    if d == -1:
        short_model = model[a:]
    else:
        short_model = model[a:d]
    return short_model

def writePrimeToFile(h,m,s,KPrime,oppFlame,Sc,alpha,delta_alpha,grid_pnts,log):
    short_model = stripModelName(model)  
    t = time.strftime("%Y-%m-%d %H-%M-%S", time.localtime())
    logFile = 'ESR_Results/{0} {1} {2}.txt'.format(t,short_model,name)
    file1 = open(logFile,'a')
    file1.write('{0}\n'.format(t))
    file1.write('Premixed twin axisemetric Extinction Strain Rate script\n')
    file1.write('Elapsed time: %02d:%02d:%02d\n'%(h, m, s))
    file1.write('\nComposition: {0}\n'.format(composition))
    file1.write('\nParameters at the extinction point:\n')
    file1.write('EXTINCTION STRAIN RATE = %.1f 1/s\n'%(KPrime))
    file1.write('Tmax =                   %.1f K\n'%(np.max(oppFlame.T)))
    file1.write('P =                      %.1f bar\n'%(oppFlame.P / 1e5))
    file1.write('Axial velocity =         %.1f cm/s\n'%(np.max(oppFlame.u)*100))
    file1.write('Consumption Speed:       %.1f cm/s\n'%(100*Sc))
    file1.write('\nCalculation parameters:\n')
    file1.write('Last alpha:             {0}\n'.format(alpha[n_last_burning]))
    file1.write('Last delta alpha:       {0}'.format(delta_alpha))
    file1.write('\n\nUser parameters:\n')
    file1.write('Mechanism =              %s\n'%(short_model))
    file1.write('Initial axial velocity = %.0f cm/s\n'%(initialAxialVelocity))
    file1.write('Inlet T & P =            %d K, %d bar\n'%(To, Po / 1e5))
    file1.write('Half width =             %.2f cm\n'%(half_width))
    if grid_pnts == 0:
        file1.write('Grid points =            automatically determined\n')
    else:
        file1.write('Grid points =            %d\n'%(grid_pnts))
    file1.write('Refine =                 {0}\n'.format(refine))
    file1.write('Transport =              %s\n'%(tran))
    file1.write('Extinction T limit =     %d K\n'%(temperature_limit_extinction))
    file1.write('Delta T min (conv)=      %.2f K\n'%(delta_T_min))
    file1.write('Initial alpha =          %.2f\n'%(u_alpha[0]))
    file1.write('Initial delta alpha =    %.2f\n'%(u_delta_alpha))
    file1.write('Delta alpha factor =     %d\n'%(delta_alpha_factor))
    file1.write('Delta alpha min (conv) = %.10f\n'%(delta_alpha_min))
    file1.write('Ratio =                  %.2f\n'%(ratio))
    file1.write('Slope =                  %.2f\n'%(slope))
    file1.write('Curve =                  %.2f\n'%(curve))
    file1.write('Prune =                  %.2f\n'%(prune))
    file1.write('SA perturbation          %.2f\n'%(dk))
    if len(log) > 0:
        file1.write('\n\nSummary of troubleshooting attempts:\n')
        for i in range(len(log)):
            file1.write('%d. \n'%(i+1))
            file1.write(log[i])
    file1.close()
    if verboseLevel >= 2:
        print('Solution saved to file.\n')
    return logFile

def appendLog(logFile,log,r,rxn):
    file1 = open(logFile,'a')
    file1.write('\nSA troubleshooting for reaction number %d, %s:\n'%(r,rxn))
    for i in range(len(log)):
        file1.write('%d.   %s\n'%(i+1,log[i]))
    file1.close()

def logSA(logFile,SA,rxnList):
    file1 = open(logFile,'a')
    file1.write('\n\nSA DATA:\n')
    file1.write("#     n-SA coef,  reaction\n")
    file1.write("--------------------------\n")
    for i in range(len(rxnList)):
        file1.write("%03d  %10.2e,  %s\n"%(i,SA[i],rxnList[i]))
    file1.close()
    
def plotFigs(oppFlame,strainRatePoint,n,u0,K_tot,T_max,file_name,n_last_burning):
    
    if (species1 == '') & (species2 ==''):
        plt.figure(figsize=(12,12))
        sub=3
    else:
        plt.figure(figsize=(12,18))
        sub=4
    plt.subplot(sub,2,1)
    plt.plot(oppFlame.grid*100, oppFlame.u*100, 'r-o', lw=2)
    plt.xlim(oppFlame.grid[0]*100, oppFlame.grid[-1]*100)
    plt.ylim(0, max(oppFlame.u)*105)
    plt.xlabel('Distance (cm)')
    plt.ylabel('Axial Velocity (cm/s)')
    plt.title('Axial velocity profile')
    plt.plot(oppFlame.grid[strainRatePoint]*100, oppFlame.u[strainRatePoint]*100,'gs')
    plt.annotate('ESR point',
                xy=(oppFlame.grid[strainRatePoint]*100, oppFlame.u[strainRatePoint]*100),
                xytext=(oppFlame.grid[-1]*10, max(oppFlame.u)*10),
                arrowprops={'arrowstyle':'->'})
    plt.subplot(sub,2,2)
    plt.plot(oppFlame.grid*100, oppFlame.T, 'g-o', lw=2)
    plt.xlim(oppFlame.grid[0]*100, oppFlame.grid[-1]*100)
    plt.ylim(0, 2000)
    plt.xlabel('Distance (cm)')
    plt.ylabel('Temperature (K)')
    plt.title('T Profile for n = %02d'%(n_last_burning))
    plt.subplot(sub,2,3)
    plt.plot(K_tot, T_max,'b-o', lw=2)
    plt.xlabel('ESR (1/s)')
    plt.ylabel(r'$T_{max}$ (K)')
    plt.title('Tmax vs. ESR')
    plt.subplot(sub,2,4)
    plt.plot(u0, T_max, 'k-o', lw=2)
    plt.xlabel('Axial Velocity (cm/s)')
    plt.ylabel('Tmax (K)')
    plt.title('Tmax vs. axial velocity')
    plt.subplot(sub,2,5)
    plt.plot(n, K_tot,'b-o', lw=2)
    plt.xlim(0, n[-1]+0.5)
    plt.xlabel('Iteration number')
    plt.ylabel('Strain rate (1/s)')
    plt.title('Strain rate vs. iteration')
    plt.plot(n[-1], K_tot[-1],'go')
    plt.annotate('ESR point',
                xy=(n[-1], K_tot[-1]),
                xytext=(n[-1]*0.7, K_tot[-1]*0.6),
                arrowprops={'arrowstyle':'->'})
    plt.subplot(sub,2,6) 
    plt.plot(n, T_max, 'k-o', lw=2)
    plt.xlim(0, n[-1]+0.5)
    plt.xlabel('Iteration number')
    plt.ylabel('Tmax (K)')
    plt.title('Tmax vs. iteration')
    if sub == 4:
        plt.subplot(sub,2,7)
        plt.plot(oppFlame.grid*100,oppFlame.X[gas.species_index(species1)])
        plt.xlabel('Distance (cm)')
        plt.ylabel('Mole fraction')
        plt.title('Concentration profile for %s'%(species1))
        plt.subplot(sub,2,8)
        plt.plot(oppFlame.grid*100,oppFlame.X[gas.species_index(species2)])
        plt.xlabel('Distance (cm)')
        plt.ylabel('Mole fraction')
        plt.title('Concentration profile for %s'%(species2))
    plt.tight_layout
    if verboseLevel >= 3:
        plt.show()
    fig_name = file_name[0:-3] + str('png')
    plt.savefig(fig_name, dpi = 100)
    if verboseLevel >= 2:
        print('\nFigure saved to file.')

def extractTopSA(logFile,SA,rxnList,d,h,m,s):
    num1 = np.linspace(0,len(SA)-1,len(SA))
    num1 = list(zip(SA,num1))
    num1.sort(key=lambda x: abs(x[0]), reverse=True)
    SA_sorted = []
    rxns_sorted = []
    for i in range(topSA):
        SA_sorted.append(SA[int(num1[i][1])])
        rxns_sorted.append(rxnList[int(num1[i][1])])
    print("\n\n\n Sorted Top SA coefficients:")
    file1 = open(logFile,'a')
    file1.write('\n\n\n Sorted Top SA coefficients:')
    file1.write("\n#     n-SA coef,  reaction")
    file1.write("\n--------------------------\n")
    for i in range(topSA):
        print("%03d  %10.2e,  %s"%(i,SA_sorted[i],rxns_sorted[i]))
        file1.write("%03d  %10.2e,  %s\n"%(i,SA_sorted[i],rxns_sorted[i]))
    file1.write("\n\nTotal elapsed time with SA: %02d days and %02d:%02d:%02d"%(d, h, m, s))
    print("\n\nTotal elapsed time with SA: %02d days and %02d:%02d:%02d"%(d, h, m, s))
    file1.close()
    return SA_sorted, rxns_sorted
        
#graphical definitions
plt.rcParams['figure.autolayout'] = True
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (8,6)

# Create directories for output files
data_directory = u_data_directory
if not os.path.exists(data_directory):
    os.makedirs(data_directory)
results_directory = 'ESR_results/'
if not os.path.exists(results_directory):
    os.makedirs(results_directory)
    
# Correct user input
if initialAxialVelocity <= 0:
    initialAxialVelocity = 10
    print("initialAxialVelocity is not allowed to be 0 or less. Was auto-set to 10 cm/s")
if (grid_pnts == 1) | (grid_pnts < 0):
    grid_pnts == 0
    print("grid_pnts parameter is not allowed to be 1 or negative. Was auto-set to 0 (automatic)")
    
print("OK")

OK


# Main

In [6]:
t0 = time.time()   # set timer
# initialize parameters from user input
gas, oppFlame, alpha, delta_alpha, width = initializer(initialAxialVelocity)

KPrime,strainRatePoint,log,n_last_burning,alpha,delta_alpha,grid_pnts,SAalpha,SAd_alpha,tracker = solver(
    gas, oppFlame, alpha, delta_alpha, width, t0)

Sc = computeConsumptionSpeed(oppFlame)

t1 = time.time() - t0
m, s = divmod(t1, 60)
h, m = divmod(m, 60)
d, h = divmod(h, 24)
if verboseLevel >= 1:
    printResults(KPrime, oppFlame, gas.density, Sc, d, h, m, s)

logFile = writePrimeToFile(h, m, s, KPrime, oppFlame, Sc, alpha, delta_alpha, grid_pnts,log)

plotFigs(oppFlame,strainRatePoint,column(tracker, 0),column(tracker, 1),column(tracker, 2),column(tracker, 3),
         logFile,n_last_burning)  # Tracker: n, u0, ESR, Tmax

if topSA > 0:
    rxnList = ct.Reaction.listFromFile(model)
    SA = [] # the normalized SA coefficients
    for i in range(gas.n_reactions):
        if verboseLevel >= 2:
            print("\nConducting SA for rxn %d out of %d: %s\n"%(
                    i, gas.n_reactions-1, rxnList[i]))
        gas, oppFlame, alpha, delta_alpha, width = initializer(initialAxialVelocity)
        gas.set_multiplier(1.0)      # reset all multipliers
        APrime = gas.forward_rate_constants[i]
        gas.set_multiplier(dk, i)    # perturb reaction i by a factor of dk
        K, log = SAsolver(
            gas, oppFlame, SAalpha, SAd_alpha, width, t0)
        if len(log) >= 1:
            appendLog(logFile,log,i,rxnList[i])
        dlnKdlnAi = ((K - KPrime) / (gas.forward_rate_constants[i] - APrime)) * (APrime / KPrime)
        if verboseLevel >= 2:
            print("\n**** ... done. Rxn %03d  '%s' has a normalized SA coefficient of %10.2e ****\n"%(
                    i,rxnList[i],dlnKdlnAi))
        SA.append(dlnKdlnAi)
    logSA(logFile,SA,rxnList)

    t2 = time.time() - t0
    m, s = divmod(t2, 60)
    h, m = divmod(m, 60)
    d, h = divmod(h, 24)

    SA_sorted, rxns_sorted = extractTopSA(logFile,SA,rxnList,d,h,m,s)

# plot topSA + legend

print("Done.")

Solving xo1247R with u0 =  200.0 cm/s:
n =   8,     u =    600.0 cm/s,     K =   1667.4 1/s,     Tmax =   1788.3 K,     elaped time: 00 00:00:28
Flame extinguished at n =   9. Restoring n =   8 with alpha = 9.00e+00 and delta_alpha = 2.00e-02 (E2)
n =  38,     u =    619.0 cm/s,     K =   1715.4 1/s,     Tmax =   1770.2 K,     elaped time: 00 00:02:28
Error: Did not converge at n =  39. alpha = 9.60e+00,   delta_alppha = 2.00e-02,   Tmax =   1769.1 K
n =  39,     u =    619.7 cm/s,     K =   1717.0 1/s,     Tmax =   1769.1 K,     elaped time: 00 00:03:04
Error: Did not converge at n =  40. alpha = 9.62e+00,   delta_alppha = 2.00e-02,   Tmax =   1767.8 K
n =  43,     u =    622.3 cm/s,     K =   1723.4 1/s,     Tmax =   1762.8 K,     elaped time: 00 00:06:50
Flame extinguished at n =  44. Restoring n =  43 with alpha = 9.68e+00 and delta_alpha = 4.00e-04 (E2)
n =  45,     u =    622.3 cm/s,     K =   1723.2 1/s,     Tmax =   1762.8 K,     elaped time: 00 00:08:28
Flame (almost) extingui

<IPython.core.display.Javascript object>


Figure saved to file.

Conducting SA for rxn 0 out of 73: CH3(18) + H(4) (+M) <=> CH4(1) (+M)

n =   2,     u =    600.0 cm/s,     K =   1666.6 1/s,     Tmax =   1732.9 K,     elaped time: 00 00:08:43
Flame extinguished at n =   3. Restoring n =   2 with alpha = 9.00e+00 and delta_alpha = 2.00e-02 (E2)
n =  25,     u =    614.5 cm/s,     K =   1702.1 1/s,     Tmax =   1747.5 K,     elaped time: 00 00:10:19
Error: Did not converge at n =  26. alpha = 9.46e+00,   delta_alppha = 2.00e-02,   Tmax =   1771.1 K
n =  31,     u =    618.4 cm/s,     K =   1711.7 1/s,     Tmax =   1760.2 K,     elaped time: 00 00:12:48
Flame extinguished at n =  32. Restoring n =  31 with alpha = 9.56e+00 and delta_alpha = 4.00e-04 (E2)

Flame extinguished at n =  33. Restoring n =  31 with alpha = 9.56e+00 and delta_alpha = 8.00e-06 (E2)
n =  34,     u =    618.4 cm/s,     K =   1710.9 1/s,     Tmax =   1760.2 K,     elaped time: 00 00:17:17
Flame (almost) extinguished at n = 34. Abortion criterion satisfied.


AttributeError: 'zip' object has no attribute 'sort'

In [12]:
SA_sorted, rxns_sorted = extractTopSA(logFile,SA,rxnList,d,h,m,s)




 Sorted Top SA coefficients:
000   -1.48e-01,  HCO(14) + OH(6) <=> CO(12) + H2O(8)
001   -1.06e-01,  H(4) + HCO(14) <=> CO(12) + H2(7)
002   -8.76e-02,  CH2(17) + OH(6) <=> CH2O(19) + H(4)
003   -8.53e-02,  CH3(18) + O(5) <=> CH2O(19) + H(4)
004    7.10e-02,  CH3(18) + O(5) <=> CO(12) + H(4) + H2(7)
005   -6.29e-02,  HCO(14) + O(5) <=> CO(12) + OH(6)
006   -6.22e-02,  CH3(18) + CH3O(26) <=> CH2O(19) + CH4(1)
007   -6.22e-02,  CH3(18) + HCO(14) <=> CH4(1) + CO(12)
008   -4.83e-02,  HCO(14) + O(5) <=> CO2(13) + H(4)
009   -4.63e-02,  CH3O(26) + OH(6) <=> CH2O(19) + H2O(8)


Total elapsed time with SA: 00 days and 09:32:37


# Main – run a case study

In [5]:
# improve troubleshooting

case = [[0.5,'nphi=0.32',0.32,'CH4(1):0.2353, O2(2):21, N2(3):79'],
       [1,'nphi=0.33',0.33,'CH4(1):0.2463, O2(2):21, N2(3):79'],
       [2,'nphi=0.34',0.34,'CH4(1):0.2576, O2(2):21, N2(3):79'],
       [3,'nphi=0.35',0.35,'CH4(1):0.2692, O2(2):21, N2(3):79'],
       [5,'nphi=0.36',0.36,'CH4(1):0.2813, O2(2):21, N2(3):79'],
       [10,'nphi=0.37',0.37,'CH4(1):0.2937, O2(2):21, N2(3):79'],
       [15,'nphi=0.38',0.38,'CH4(1):0.3065, O2(2):21, N2(3):79'],
       [30,'nphi=0.39',0.39,'CH4(1):0.3197, O2(2):21, N2(3):79'],
       [50,'nphi=0.40',0.40,'CH4(1):0.3333, O2(2):21, N2(3):79'],
       [70,'nphi=0.41',0.41,'CH4(1):0.3475, O2(2):21, N2(3):79'],
       [100,'nphi=0.42',0.42,'CH4(1):0.3621, O2(2):21, N2(3):79'],
       [110,'nphi=0.43',0.43,'CH4(1):0.3772, O2(2):21, N2(3):79'],
       [120,'nphi=0.44',0.44,'CH4(1):0.3929, O2(2):21, N2(3):79'],
       [140,'nphi=0.45',0.45,'CH4(1):0.4091, O2(2):21, N2(3):79'],
       [160,'nphi=0.46',0.46,'CH4(1):0.4259, O2(2):21, N2(3):79'],
       [180,'nphi=0.47',0.47,'CH4(1):0.4434, O2(2):21, N2(3):79'],
       [190,'nphi=0.48',0.48,'CH4(1):0.4615, O2(2):21, N2(3):79'],
       [195,'nphi=0.49',0.49,'CH4(1):0.4804, O2(2):21, N2(3):79'],
       [200,'nphi=0.50',0.50,'CH4(1):0.5000, O2(2):21, N2(3):79'],
       [150,'nphi=0.51',0.51,'CH4(1):0.5204, O2(2):21, N2(3):79'],
       [130,'nphi=0.52',0.52,'CH4(1):0.5417, O2(2):21, N2(3):79'],
       [100,'nphi=0.53',0.53,'CH4(1):0.5638, O2(2):21, N2(3):79'],
       [80,'nphi=0.54',0.54,'CH4(1):0.5870, O2(2):21, N2(3):79'],
       [40,'nphi=0.55',0.55,'CH4(1):0.6111, O2(2):21, N2(3):79'],
       [20,'nphi=0.56',0.56,'CH4(1):0.6364, O2(2):21, N2(3):79'],
       [15,'nphi=0.57',0.57,'CH4(1):0.6628, O2(2):21, N2(3):79'],
       [10,'nphi=0.58',0.58,'CH4(1):0.6905, O2(2):21, N2(3):79'],
       [7,'nphi=0.59',0.59,'CH4(1):0.7195, O2(2):21, N2(3):79'],
       [4,'nphi=0.60',0.60,'CH4(1):0.7500, O2(2):21, N2(3):79'],
       [2,'nphi=0.61',0.61,'CH4(1):0.7821, O2(2):21, N2(3):79'],
       [1,'nphi=0.62',0.62,'CH4(1):0.8158, O2(2):21, N2(3):79'],
       [0.5,'nphi=0.63',0.63,'CH4(1):0.8514, O2(2):21, N2(3):79']]


nphi = []
Kfinal = []

for icase in case:
    
    print("\n")
    print(icase[0])
    print(icase[1])
    print(icase[2])
    print("\n")
    
    
    initialAxialVelocity = icase[0]
    name = icase[1]
    nphi.append(icase[2])
    composition = icase[3]

    t0 = time.time()   # set timer
    # initialize parameters from user input
    gas, oppFlame, alpha, delta_alpha, width = initializer(initialAxialVelocity)

    KPrime,K_tot,strainRatePoint,log,T_max,n_last_burning,alpha,delta_alpha,grid_pnts,SAalpha,SAd_alpha,u0 = solver(
        gas, oppFlame, alpha, delta_alpha, width, t0)

    kfinal.append(K)
    
    Sc = computeConsumptionSpeed(oppFlame)

    t1 = time.time() - t0
    m, s = divmod(t1, 60)
    h, m = divmod(m, 60)
    if verboseLevel >= 1:
        printResults(KPrime, oppFlame, gas.density, Sc, h, m, s)

    logFile = writePrimeToFile(h, m, s, KPrime, oppFlame, Sc, alpha, delta_alpha, grid_pnts,log)
    plotFigs(oppFlame,strainRatePoint,K_tot,T_max,u0,logFile,n_last_burning)  
    
print("Done.")



1
nphi=0.40
0.4


Solving xo1247-C2i with u0 =    1.0 cm/s:

Flame extinguished at n =   0. Restoring n =   0 with alpha = 1.00e+00 and delta_alpha = 2.00e-02 (E2)

Couldn't ignite. Changing initialAxialVelocity from    1.0 to    0.5 at n=0 (TrSh2)

Flame extinguished at n =   1. Restoring n =   0 with alpha = 1.00e+00 and delta_alpha = 2.00e-02 (E2)

Couldn't ignite. Changing initialAxialVelocity from    0.5 to    0.3 at n=0 (TrSh2)

Flame extinguished at n =   1. Restoring n =   0 with alpha = 1.00e+00 and delta_alpha = 2.00e-02 (E2)

Couldn't ignite. Changing initialAxialVelocity from    0.3 to    0.3 at n=0 (TrSh2)

Flame extinguished at n =   1. Restoring n =   0 with alpha = 1.00e+00 and delta_alpha = 2.00e-02 (E2)

Couldn't ignite. Changing initialAxialVelocity from    0.3 to    0.3 at n=0 (TrSh2)


KeyboardInterrupt: 

## INSTRUCTIONS
Convert your mechanism into a .cti format using the <a href="http://www.cantera.org/docs/sphinx/html/cti/input-files.html#sec-ck-format-conversion">ck2cti</a> script.<BR>
Then, run this ESR solver for your case, and try to get it to converge at a minimum time without SA, mainly by optimizing the axial velocity<BR> (by setting topSA to 0).<BR>
Once no auto-troubleshooting messages appear (i.e., "Couldn't ignite. Changing initialAxialVelocity..."; note that the "Flame extinguished at n = x. Restoring n = y" messages should always always appear), start with the sensitivity study by setting topSA to the desired value.


### SOME GENERAL ALGORITHM COMMENTS:

The velocity is multiplied each iteration by a `strain factor`:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$v = v \times strain Factor$<BR>
$\alpha$ is an auxilary parameter defined as:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha_{i+1} = \alpha_i + \Delta\alpha$<BR>
initialized with the following default values<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha_0 = 1$, and $\Delta\alpha = 1$<BR>
and results in the following series:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha = [1, 2, 3, 4...]$<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$strain Factor = [2, 3/2, 4/3...]$,<BR>
since the `strain factor` is redefined each iteration as:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$strain Factor = \alpha_i / \alpha _{i-1}$<BR>
Once extinction occurs, however, the last burning solution is restored and $\Delta\alpha$ now gets divided by $\Delta\alpha Factor$ with a default value of 50:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\Delta\alpha = \Delta\alpha / 50$<BR>
causing the $\alpha$ and `strain factor` series to advance in smaller steps:<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha = [1, 2, 3, 4, 4.02, 4.04, 4.06...]$<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$strain Factor = [2, 3/2, 4/3, 4.02/4, 4.04/4.02, 4.06/4.04...]$<BR>
So if $\Delta\alpha Factor$ = 50, $\Delta\alpha$ will get the following values: 1, 2E-2, 4E-4, 8E-6, 1.6E-7, etc...<BR>
Convergence creteria are both $\Delta T < \Delta T _{min}$ and $\Delta\alpha < \Delta\alpha_{min}$<BR>


### CRITERIA FOR GRID REFINEMENT:

ratio: &nbsp;additional points will be added if the ratio of the spacing on<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;either side of a grid point exceeds this value<BR>
slope: Adds points in regions of high slope.<BR>
curve: Adds points in regions of high curvature.<BR>
prune: if the slope or curve criteria are satisfied to the level of<BR>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;'prune', the grid point is assumed not to be needed and is removed.<BR>


## Installing the latest version of Cantera:

Open CMD and type:<BR>
`Conda create –n canteraEnv python=3 jupyter`<BR>
`activate canteraEnv`<BR>
`conda install -c https://conda.anaconda.org/cantera/label/dev cantera`<BR>
`python -m ipykernel install --user --name canteraEnv --display-name "Cantera-Py3"`<BR>

Make sure you activate the above environment each time you want to install relevant dependencies.